<a href="https://colab.research.google.com/github/AlinaStudyPM/UniversityML/blob/main/MyML_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
!pip install haversine

In [71]:
!pip install folium

In [72]:
import pandas as pd
import numpy as np
from haversine import haversine
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
import folium

In [6]:
data = pd.read_csv('ALL_SPB_STOPS.csv', encoding='CP1251', delimiter=';')
data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7154 entries, 0 to 7153
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ID_STOP    7154 non-null   int64  
 1   STOP_NAME  7154 non-null   object 
 2   LATITUDE   7154 non-null   float64
 3   LONGITUDE  7154 non-null   float64
dtypes: float64(2), int64(1), object(1)
memory usage: 223.7+ KB


In [19]:
metro_stops = data[data['ID_STOP'].astype(str).str.startswith('100')]
metro_stops

,ID_STOP,STOP_NAME,LATITUDE,LONGITUDE
0,100101,Пр.Ветеранов-1,59.841604,30.253984
1,100102,Пр.Ветеранов-2,59.842042,30.250084
2,100103,Ленинский пр.-1,59.851683,30.268385
3,100104,Ленинский пр.-2,59.849710,30.268348
4,100105,Автово,59.867292,30.261133
...,...,...,...,...
70,100513,Чкаловская,59.960987,30.292046
71,100514,Крестовский остров,59.971792,30.259414
72,100515,Старая Деревня,59.989538,30.255218
73,100516,Комендантский пр.,60.007810,30.259476


In [20]:
auto_stops = data[~data['ID_STOP'].astype(str).str.startswith('100')]
auto_stops

,ID_STOP,STOP_NAME,LATITUDE,LONGITUDE
75,1282,"10 КМ [145, 165, 482, 482а, 482в, 484, 484б]",59.773372,30.125111
76,1285,12-13 ЛИНИИ (ПО СРЕДНЕМУ ПР. В.О.) [6],59.941121,30.271757
77,1287,18-19 ЛИНИИ [6],59.938798,30.264141
78,1290,"1-Я И КАДЕТСКАЯ ЛИНИЯ,УГ. СРЕДНЕГО ПР. [6]",59.944538,30.287008
79,1291,1-Я КРАСНОАРМЕЙСКАЯ УЛ. [71],59.915965,30.310436
...,...,...,...,...
7149,4997,ЗАВОД ИМ. КАЛИНИНА [тест] <,59.950695,30.266132
7150,4998,МАЛЫЙ ПР. В.О. ПО 9 ЛИНИИ [тест] <,59.945698,30.271969
7151,5003,АДМИРАЛТЕЙСКИЙ ПР. [тест] <,59.936026,30.309259
7152,7069,ПР. ЛУНАЧАРСКОГО(тест),60.031439,30.392572


In [47]:
metro_name = "Маяковская"
radius = 1000
metro_stop_info = metro_stops[metro_stops['STOP_NAME'].str.contains(metro_name)]
metro_stop_info

,ID_STOP,STOP_NAME,LATITUDE,LONGITUDE
47,100304,Маяковская,59.931592,30.355142


In [48]:
if not metro_stop_info.empty:
  latitude_metro = metro_stop_info['LATITUDE'].values[0]
  longitude_metro = metro_stop_info['LONGITUDE'].values[0]
else:
  print("ОШИБКА: Не удалось найти станцию метро с таким названием.")

In [49]:
def calculate_distance(row):
  return 1000 * haversine((latitude_metro, longitude_metro), (row['LATITUDE'], row['LONGITUDE']))

In [50]:
auto_stops['Distance'] = auto_stops.apply(calculate_distance, axis=1)
auto_stops

<ipython-input-50-80e5da2834fd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  auto_stops['Distance'] = auto_stops.apply(calculate_distance, axis=1)


,ID_STOP,STOP_NAME,LATITUDE,LONGITUDE,Distance
75,1282,"10 КМ [145, 165, 482, 482а, 482в, 484, 484б]",59.773372,30.125111,21784.100714
76,1285,12-13 ЛИНИИ (ПО СРЕДНЕМУ ПР. В.О.) [6],59.941121,30.271757,4764.235845
77,1287,18-19 ЛИНИИ [6],59.938798,30.264141,5132.274068
78,1290,"1-Я И КАДЕТСКАЯ ЛИНИЯ,УГ. СРЕДНЕГО ПР. [6]",59.944538,30.287008,4059.018984
79,1291,1-Я КРАСНОАРМЕЙСКАЯ УЛ. [71],59.915965,30.310436,3037.369714
...,...,...,...,...,...
7149,4997,ЗАВОД ИМ. КАЛИНИНА [тест] <,59.950695,30.266132,5393.442469
7150,4998,МАЛЫЙ ПР. В.О. ПО 9 ЛИНИИ [тест] <,59.945698,30.271969,4891.129809
7151,5003,АДМИРАЛТЕЙСКИЙ ПР. [тест] <,59.936026,30.309259,2603.194173
7152,7069,ПР. ЛУНАЧАРСКОГО(тест),60.031439,30.392572,11296.115270


In [51]:
nearby_auto_stops = auto_stops[auto_stops['Distance'] < radius]
nearby_auto_stops

,ID_STOP,STOP_NAME,LATITUDE,LONGITUDE,Distance
140,1377,"БКЗ ""ОКТЯБРЬСКИЙ""",59.936895,30.364611,791.183861
242,1476,"ДВОРЕЦ ТВОРЧЕСТВА ЮНЫХ [7, 3, 27, 22, 24]",59.933590,30.339277,911.337175
290,14902,"Пять углов (3, 8, 15)",59.926304,30.343023,895.383815
294,14907,"Владимирская площадь (3, 8, 15)",59.929517,30.347792,470.025637
296,14909,"Невский проспект (3, 8, 15)",59.933391,30.348150,437.907500
...,...,...,...,...,...
6882,4217,ЛИТЕЙНЫЙ ПР. (тест),59.933371,30.348068,440.952260
6883,4218,МАРИИНСКАЯ БОЛЬНИЦА (тест),59.934743,30.348238,520.295171
6886,4223,"УЛ. НЕКРАСОВА, 1",59.938938,30.349131,882.813789
6887,4243,УЛ. ВОССТАНИЯ (тест),59.938729,30.360935,856.725814


In [54]:
coords = nearby_auto_stops[['LATITUDE', 'LONGITUDE']].values

In [106]:
number_of_clusters_kmeans = 13

model_kmeans = KMeans(n_clusters=number_of_clusters_kmeans, random_state=42)
clusters_kmeans = model_kmeans.fit_predict(coords)

nearby_auto_stops['KMeans_Cluster'] = clusters_kmeans

<ipython-input-106-ab7062d02c2e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nearby_auto_stops['KMeans_Cluster'] = clusters_kmeans


In [123]:
model_dbscan = DBSCAN(eps=0.005, min_samples=5)
clusters_dbscan = model_dbscan.fit_predict(coords)

nearby_auto_stops['DBSCAN_Cluster'] = clusters_dbscan

nearby_auto_stops

<ipython-input-123-c7fecd8a96a3>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nearby_auto_stops['DBSCAN_Cluster'] = clusters_dbscan


,ID_STOP,STOP_NAME,LATITUDE,LONGITUDE,Distance,KMeans_Cluster,DBSCAN_Cluster
140,1377,"БКЗ ""ОКТЯБРЬСКИЙ""",59.936895,30.364611,791.183861,9,0
242,1476,"ДВОРЕЦ ТВОРЧЕСТВА ЮНЫХ [7, 3, 27, 22, 24]",59.933590,30.339277,911.337175,4,-1
290,14902,"Пять углов (3, 8, 15)",59.926304,30.343023,895.383815,11,0
294,14907,"Владимирская площадь (3, 8, 15)",59.929517,30.347792,470.025637,5,0
296,14909,"Невский проспект (3, 8, 15)",59.933391,30.348150,437.907500,1,0
...,...,...,...,...,...,...,...
6882,4217,ЛИТЕЙНЫЙ ПР. (тест),59.933371,30.348068,440.952260,1,0
6883,4218,МАРИИНСКАЯ БОЛЬНИЦА (тест),59.934743,30.348238,520.295171,1,0
6886,4223,"УЛ. НЕКРАСОВА, 1",59.938938,30.349131,882.813789,8,0
6887,4243,УЛ. ВОССТАНИЯ (тест),59.938729,30.360935,856.725814,2,0


In [109]:
map_stops_kmeans = folium.Map(
    location=[latitude_metro, longitude_metro],
    zoom_start=15,
)

colors = [
    'red',
    'blue',
    'gray',
    'lightred',
    'orange',
    'beige',
    'green',
    'darkgreen',
    'lightgreen',
    'darkblue',
    'lightblue',
    'purple',
    'darkpurple',
    'pink',
    'cadetblue',
    'lightgray',
    'black',
    'darkred'
]

for i in range(number_of_clusters_kmeans):
  cluster_data = nearby_auto_stops[nearby_auto_stops['KMeans_Cluster'] == i]
  for _, row in cluster_data.iterrows():
    folium.Marker(
        location=(row['LATITUDE'], row['LONGITUDE']),
        popup = 'Кластер ' + str(row['KMeans_Cluster']) + ': ' +row['STOP_NAME'],
        icon=folium.Icon(color=colors[i])
        ).add_to(map_stops_kmeans)
map_stops_kmeans.save('spb_agglomeration_map.html')

In [124]:
map_stops_dbscan = folium.Map(
    location=[latitude_metro, longitude_metro],
    zoom_start=15,
)

for cluster_label in nearby_auto_stops['DBSCAN_Cluster'].unique():
        if cluster_label != -1:  # Пропускаем шумовые точки, если есть
            cluster_data = nearby_auto_stops[nearby_auto_stops['DBSCAN_Cluster'] == cluster_label]
            for _, row in cluster_data.iterrows():
              folium.Marker(
                  location=(row['LATITUDE'], row['LONGITUDE']),
                  popup = 'Кластер ' + str(cluster_label) + ': ' +row['STOP_NAME'],
                  icon=folium.Icon(color=colors[i])
              ).add_to(map_stops_dbscan)
map_stops_dbscan.save('spb_agglomeration_map.html')